In [1]:
! pip install wikipedia

# Import Libraries

In [2]:
import wikipedia
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import wikipedia
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import numpy as np
import warnings 
warnings.filterwarnings("ignore")

2024-07-09 16:53:52.497331: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 16:53:52.497384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 16:53:52.498868: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Download NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# English Generation

In [4]:
wikipedia.set_lang("en")  # Set language to English
page_content = wikipedia.page("Python (programming language)").content


tokenizer = Tokenizer()
tokenizer.fit_on_texts([page_content])
total_words = len(tokenizer.word_index) + 1

# Convert text to sequences
input_sequences = []
for line in page_content.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors, label = input_sequences[:,:-1],input_sequences[:,-1]


model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(predictors, label, epochs=50, verbose=1)


seed_text = "Python is"
next_words = 150

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word


Epoch 1/50
 10/207 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.0000e+00 - loss: 7.5076

I0000 00:00:1720544039.765303     731 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720544039.786993     731 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


207/207 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.0250 - loss: 6.9116
Epoch 2/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.0454 - loss: 6.2237
Epoch 3/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.0591 - loss: 6.0162
Epoch 4/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.0590 - loss: 6.2792
Epoch 5/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.1090 - loss: 5.3737
Epoch 6/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.1317 - loss: 5.0331
Epoch 7/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.1568 - loss: 4.7632
Epoch 8/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.2035 - loss: 4.3838
Epoch 9/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.2622 - loss: 3.9756
Epoch 10/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.3161 - loss: 3.6736
Epoch 11/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.3736 - loss: 3.3469
Epoch 12/50
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/ste

In [5]:
print(seed_text)

Python is a multi paradigm programming language object oriented programming and structured programming are fully supported and many of their features support functional programming and aspect oriented programming including metaprogramming and metaobjects many other paradigms are supported via extensions including design by contract and logic programming language and structured programming are fully supported and many of their use the operator to the language software parts of the typing module provides decimal floating point division e g 5 2 2 5 and is 9 series 3 8 was just in time it also also usable with python community direct pythonic which is a statement which is a value at a time jit and an experimental bytecode compiler and removals from the c api some standard library modules and many deprecated classes functions and methods will be removed in python 3 15 and or 3 16 starting with 3 13 it and later versions


In [6]:
# # Retrieve content from Arabic Wikipedia
# wikipedia.set_lang("ar")  # Set language to Arabic
# page_content = wikipedia.page("برمجة بيثون").content  # Example Arabic Wikipedia page

# Arabic Generation

In [7]:
# Retrieve content from Arabic Wikipedia
wikipedia.set_lang("ar")  # Set language to Arabic
page_content = wikipedia.page("صلاح الدين").content  # Example Arabic Wikipedia page

tokenizer = Tokenizer()
tokenizer.fit_on_texts([page_content])
total_words = len(tokenizer.word_index) + 1

# Convert text to sequences
input_sequences = []
for line in page_content.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors, label = input_sequences[:,:-1],input_sequences[:,-1]


model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(predictors, label, epochs=50, verbose=1)


seed_text = "صلاح الدين هو"
next_words = 150

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word


Epoch 1/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.0308 - loss: 8.1186
Epoch 2/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.0447 - loss: 7.9733
Epoch 3/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.0541 - loss: 7.6400
Epoch 4/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.0600 - loss: 7.5306
Epoch 5/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.0519 - loss: 7.5941
Epoch 6/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.0541 - loss: 7.5061
Epoch 7/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.0584 - loss: 7.3544
Epoch 8/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.0609 - loss: 7.2104
Epoch 9/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.0616 - loss: 7.0402
Epoch 10/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.0642 - loss: 6.9473
Epoch 11/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.0646 - loss: 6.8552
Epoch 12/50
422/422 ━━━━━━━━━━

In [8]:
print(seed_text)

صلاح الدين هو من حلب ضد الجيش الصليبي عندما هاجم جناحه، وهو صاحب قيسارية كيفا، واستولت القوات المشتركة على عين جالوت وعندما تقدمت القوات الصليبية، والتي كانت كان غير أن يتفق مع المسيحيين الأرثوذكس من عرب وروم يوعدهم بالخير والعفو عنهم إذا لم يساعدوا الفرنجة في القتال وأن يسلموا المدينة لصلاح الدين من الجهة التي يسكنون بها في مصر لم يرغب صلاح الدين الامتثال لهذا الأمر خوفًا من النفوذ الشيعي في مصر، وأخذ يراوغ في تأخير الأمر، إلا أن نور الدين زنكي في مشاكله الداخلية، إضافة إلى وفاة أسد الدين شيركوه ونجم الدين أيوب وخَرَجَ بهما إلى بغداد ومن هناك نزلوا تكريت، ومات شاذي بها وعلى قبره قبة داخل البلد»، وهو ما يؤكده سعيد عبد الفتاح عاشور وجمال الدين الشيال وعبد المنعم ماجد في حين وبعد أن استقر الأيوبيون في السلطة، تبنى جماعة من ملوكهم أنسابًا عربية نبيلة وأنكروا النسب الكردي، حيث قالوا «إنما نحن عرب، نزلنا عند الأكراد وتزوجنا منهم »، ومع


In [9]:
seed_text = "صلاح الدين هو"
next_words = 1000

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━

In [10]:
print(seed_text)

صلاح الدين هو من حلب ضد الجيش الصليبي عندما هاجم جناحه، وهو صاحب قيسارية كيفا، واستولت القوات المشتركة على عين جالوت وعندما تقدمت القوات الصليبية، والتي كانت كان غير أن يتفق مع المسيحيين الأرثوذكس من عرب وروم يوعدهم بالخير والعفو عنهم إذا لم يساعدوا الفرنجة في القتال وأن يسلموا المدينة لصلاح الدين من الجهة التي يسكنون بها في مصر لم يرغب صلاح الدين الامتثال لهذا الأمر خوفًا من النفوذ الشيعي في مصر، وأخذ يراوغ في تأخير الأمر، إلا أن نور الدين زنكي في مشاكله الداخلية، إضافة إلى وفاة أسد الدين شيركوه ونجم الدين أيوب وخَرَجَ بهما إلى بغداد ومن هناك نزلوا تكريت، ومات شاذي بها وعلى قبره قبة داخل البلد»، وهو ما يؤكده سعيد عبد الفتاح عاشور وجمال الدين الشيال وعبد المنعم ماجد في حين وبعد أن استقر الأيوبيون في السلطة، تبنى جماعة من ملوكهم أنسابًا عربية نبيلة وأنكروا النسب الكردي، حيث قالوا «إنما نحن عرب، نزلنا عند الأكراد وتزوجنا منهم »، ومع ذلك اختلف هؤلاء في نسبهم فالملك المعز إسماعيل الأيوبي على اليمن أرجع نسب بني أيوب إلى بني أمية وحين بلغ ذلك الملك العادل سيف الدين غازي بن قطب الدين مودود بت